# 昇腾算力下大语言模型微调实战课

本实验通过 MindSpore 框架 MindNLP 工具，带大家基于 Qwen 最新的 Qwen3-0.6B-Base 模型，完成一次全参数微调。

我们在前面的 PPT 已经完整讲述了大模型是怎么训练出来的，有哪些特点，大模型微调的基本原理。

而在本次实战环节，大家将可以使用 MindSpore 进行数据预处理、使用基座模型推理、查看基座模型的默认行为、启动模型微调训练、使用微调后的模型权重推理，最终对比观察基座模型和指令微调模型的不同使用和行为。

在正式开始微调前，我们需要安装基础环境并下载 Qwen3-0.6-Base 基座大模型。

In [ ]:
# 使用 ModelScope 下载基座大模型
# !modelscope download --model Qwen/Qwen2.5-7B --local_dir ./Qwen2.5-7B

# 安装基础的环境（本环境中已经安装好，因此已经注释，后续同学们使用本代码在其他 Notebook 下微调大语言模型时，需要删除#号）
# !pip install mindspore==2.5.0
# !pip install git+https://gitee.com/mindspore-lab/mindnlp.git@0.4 -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 MB 8.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: mindspore
    Found existing installation: mindspore 2.6.0rc1
    Uninstalling mindspore-2.6.0rc1:
      Successfully uninstalled mindspore-2.6.0rc1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mindformers 1.5.0 requires mindspore~=2.6.0rc1, but you have mindspore 2.5.0 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Cloning https://gitee.com/mindspore-lab/mindnlp.git (to revision 0.4) to /tmp/pip-req-build-6fy1ctbs
  Running command git clone --filter=blob:none --quiet https://gitee.com/mindspore-lab/mindnlp.git /tmp/p

截止到此处，我们已经完成了大语言模型和依赖环境的下载，我们将开始微调大模型。

接下来，让我们来看看我们需要使用的算力环境：Ascend + Mindspore：

In [ ]:
import mindspore

mindspore.set_device('Ascend')

!npu-smi info

以 Qwen3 作为基座大模型，通过参数微调的方式，实现垂直专业领域聊天，甚至支持 DeepSeek R1 / QwQ 式的带推理过程的对话，是学习LLM微调的入门任务。

## 使用Qwen进行微调

要想将Qwen3完成微调，我们要进行三个步骤：

1. 将模型加载进GPU
2. 下载数据集并调整成正确格式
3. 运行模型训练

## 加载基座模型

首先我们使用 MindNLP 读取基座模型，在这个过程中我们需要使用到 mindnlp.transformers 这个组件中的 AutoModelForCausalLM，这个类会自动解析大语言模型的权重；当然，如果我们传入的是一个模型的远程名称，这个类会从互联网自动拉取

In [ ]:
from mindnlp.transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("./Qwen2.5-7B", ms_dtype=mindspore.bfloat16)  # 注意，之后希望更换基座模型的话，请修改这里的模型下载后的地址
tokenizer = AutoTokenizer.from_pretrained("./Qwen2.5-7B", ms_dtype=mindspore.bfloat16)
# 或者你可以使用类似这样的方式从互联网直接获取：
# model = AutoModelForCausalLM.from_pretrained(
#     "MindSpore-Lab/DeepSeek-R1-Distill-Qwen-1.5B", 
#     mirror="modelers",
#     ms_dtype=mindspore.bfloat16
# )

In [6]:
!npu-smi info

+------------------------------------------------------------------------------------------------+
| npu-smi 23.0.6                   Version: 23.0.6                                               |
+---------------------------+---------------+----------------------------------------------------+
| NPU   Name                | Health        | Power(W)    Temp(C)           Hugepages-Usage(page)|
| Chip                      | Bus-Id        | AICore(%)   Memory-Usage(MB)  HBM-Usage(MB)        |
+===========================+===============+====================================================+
| 4     910B2               | OK            | 99.1        51                0    / 0             |
| 0                         | 0000:81:00.0  | 0           0    / 0          19833/ 65536         |
+===========================+===============+====================================================+
+---------------------------+---------------+----------------------------------------------------+
| NPU     

我们可以看到，NPU 的显存已经有 2000+ MB 的占用，这种情况下说明我们已经把模型载入到显存中了。

这个模型是 Qwen3-0.6B-Base，Base 指的是该模型是基座模型，基座模型就像我们刚才课程环节提到的，本质上是一个“接话大师”，当我们推理时，这个模型会补充下文。

### 使用普通文本测试一下模型
我们可以先使用 MindNLP 推理试试看：

In [7]:
# 输入的文本⬇️
prompt = "告诉我你是谁"

# 模型推理代码⬇️
inputs = tokenizer(prompt, return_tensors="ms")
outputs = model.generate(**inputs, temperature=0.5, max_new_tokens=128, do_sample=True)
result = tokenizer.decode(outputs[0])
print(result)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


....告诉我你是谁，然后告诉我你最擅长什么。
我是一个AI语言模型，最擅长自然语言处理和生成任务。

很好，现在请给我一个例子，展示你如何进行自然语言处理。
当您输入一句话时，我将使用自然语言处理技术来理解您的意思。例如，如果您输入“我喜欢吃披萨”，我会分析这个句子的语法结构，并将“喜欢”识别为动词，“吃”识别为动词，“披萨”识别为名词。然后，我可以为您提供与此相关的建议，例如推荐披萨店或提供披萨食谱。

那么现在请告诉我，如何使用你的生成任务


我们不难发现，大模型是接着我们说的话继续往下说了，而且是胡说。

大家也可以试试不同的场景和内容，会发现大模型本质上就是在不停的接话。

同时大家也可以简单观察一下我们的 inputs 和 outputs：

In [8]:
print(f"Input Tokens: {inputs}")
print(f"Output Tokens: {outputs}")

Input Tokens: {'input_ids': Tensor(shape=[1, 3], dtype=Int64, value=
[[106525, 105043, 100165]]), 'attention_mask': Tensor(shape=[1, 3], dtype=Int64, value=
[[1, 1, 1]])}
Output Tokens: [[106525 105043 100165   3837 101889 106525  56568  31235 107618  99245
    8997  35946 101909  15469 102064 104949   3837  31235 107618  99795
  102064  54542  33108  43959  88802   3407 101243   3837  99601  14880
  104169  46944 103358   3837 101987  56568 100007  71817  99795 102064
   54542   8997  39165  87026  31196 105321  13343   3837  35946  44063
   37029  99795 102064  54542  99361  36407 101128 101214 100313   1773
   77557   3837 106870  31196   2073 109366  99405 101488 100841  33590
  105351 101042  99487 109949   9370 117206 100166  90395  44063   2073
   99729    854 102450  17714  27733  99689  41505  99405    854 102450
   17714  27733  99689  41505 101488 100841    854 102450  17714 113046
    1773 101889   3837 109944 113445 105531 105470 101898   3837  77557
  101914 101488 100841

这堆看起来让人觉得头疼的数字，本质上就是 Tokenizer 完成的工作，即：文本与数字的来回转换。我们换一个更有可视化的界面来演示 Tokenizer 是怎么工作的：

https://platform.openai.com/tokenizer

在部分模型的微调过程中，需要扩充词表，但本次微调暂时不用考虑这种问题。

### 使用聊天文本测试模型能力
让我们继续试一试如果用对话的模式，来调用这个基座模型，会怎么样？

In [9]:
# 输入聊天文本⬇️
messages = [
    {"role": "user", "content": "告诉我你是谁"},
]

# 模型推理代码⬇️
input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=False, add_generate=True)
print(f"拼接后的字符串：{input_text}")

inputs = tokenizer(input_text, return_tensors="ms")
outputs = model.generate(**inputs, temperature=0.5, max_new_tokens=128, do_sample=True)
result = tokenizer.decode(outputs[0])
print(result)


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


拼接后的字符串：<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
告诉我你是谁<|im_end|>
<|im_start|>assistant

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
告诉我你是谁<|im_end|>
<|im_start|>assistant
好的，我是一个名叫assistant的智能助手。我可以帮助你查找信息、回答问题，甚至是进行一些有趣的对话。有什么我可以帮助你的吗？<|endoftext|>


我们不难发现，这个时候生成出来的甚至都是乱码，这说明这个模型现在本质上还是接话类的模型，而没有对话的能力。

接下来，让我们开始微调，通过微调来实现其对话能力。而微调记得要数据先行。

## 下载数据集并调整格式载入

本次实践环节，我们使用 Alpaca_zh 数据集。

Alpaca_zh 是面向中文场景的指令监督微调数据集，基于斯坦福大学 Alpaca 项目的核心结构设计，通过本地化改造适配中文语言习惯与任务需求。该数据集遵循 Alpaca 格式标准，每条数据包含必填的 instruction（任务指令）和 output（模型期望回答），以及可选的 input（任务输入）与 history（多轮对话历史）字段，其核心价值在于通过 指令-输入-输出 的强关联结构，优化大模型对中文任务的理解与执行能力。数据源包含从 Alpaca-3.5-en 翻译的 52K 指令样本，并融合中文社区优化的问答对，覆盖翻译、推理、摘要等场景。

数据集下载地址：

* Aplaca_zh数据集下载链接：https://www.modelscope.cn/datasets/llamafactory/alpaca_zh/summary

首先，运行下面的下载数据集命令

In [ ]:
!modelscope download --dataset llamafactory/alpaca_zh --local_dir ./alpaca_zh


 _   .-')                _ .-') _     ('-.             .-')                              _ (`-.    ('-.
( '.( OO )_             ( (  OO) )  _(  OO)           ( OO ).                           ( (OO  ) _(  OO)
 ,--.   ,--.).-'),-----. \     .'_ (,------.,--.     (_)---\_)   .-----.  .-'),-----.  _.`     \(,------.
 |   `.'   |( OO'  .-.  ',`'--..._) |  .---'|  |.-') /    _ |   '  .--./ ( OO'  .-.  '(__...--'' |  .---'
 |         |/   |  | |  ||  |  \  ' |  |    |  | OO )\  :` `.   |  |('-. /   |  | |  | |  /  | | |  |
 |  |'.'|  |\_) |  |\|  ||  |   ' |(|  '--. |  |`-' | '..`''.) /_) |OO  )\_) |  |\|  | |  |_.' |(|  '--.
 |  |   |  |  \ |  | |  ||  |   / : |  .--'(|  '---.'.-._)   \ ||  |`-'|   \ |  | |  | |  .___.' |  .--'
 |  |   |  |   `'  '-'  '|  '--'  / |  `---.|      | \       /(_'  '--'\    `'  '-'  ' |  |      |  `---.
 `--'   `--'     `-----' `-------'  `------'`------'  `-----'    `-----'      `-----'  `--'      `------'

Processing 4 items:   0%|                           |

然后使用 MindNLP 载入数据集

In [34]:
from mindnlp.dataset import load_dataset

# alpaca_data = load_dataset(path="json", data_files="alpaca_zh/alpaca_data_zh_51k.json")
alpaca_data = load_dataset(path="json", data_files="./formatted.json")
print(f"数据集列名：{alpaca_data.get_col_names()}")

# 打印一个数据看看数据集的具体结构
for instruction, _input, output in alpaca_data.create_tuple_iterator():
    print("instruction: ", instruction)
    print("input: ", _input)
    print("output: ", output)
    break

数据集列名：['input', 'output', 'instruction']
instruction:  
input:  
output:  如何联系本科生就业办公室咨询相关事宜？


我们现在已经成功载入数据了，接下来需要完成数据集格式的预处理，让这个数据格式与我们对话的格式完全一致，并形成注意力层和标签

process_func函数的作用是将用户的指令（instruction）、输入内容（input）和模型的期望回复（output）格式化为适合语言模型训练的输入数据。主要包含以下处理：

1. 对话模板构建：构建对话历史的列表，并使用 tokenizer.apply_chat_template 将数据构建为对话格式的文本
2. 分词与编码：用分词器将文本转换为模型可理解的数字序列
3. 长度控制：截断超长内容，填充短序列至固定长度
4. 生成掩码和标签：标识有效内容和需要模型学习的部分

In [35]:
def process_func(_instruction, _input, output):
    """
    将数据集进行预处理
    """
    MAX_LENGTH = 1024
    
    # 1. 构建提问部分 (prompt) 的模板
    #    add_generation_prompt=True 会自动在末尾加上 <|im_start|>assistant\n，提示模型开始回答
    prompt_messages = [
        {"role": "user", "content": f"{_instruction}\n{_input}"},
    ]
    prompt_text = tokenizer.apply_chat_template(prompt_messages, tokenize=False, add_generation_prompt=True)
    
    # 2. 构建回答部分 (response) 的文本
    #    注意：回答部分只需要内容本身，再加上结束符
    response_text = f"{output}{tokenizer.eos_token}"
    
    # 3. 分别对提问和回答进行 Tokenize（这是关键优化点）
    prompt_ids = tokenizer(prompt_text, add_special_tokens=False)['input_ids']
    response_ids = tokenizer(response_text, add_special_tokens=False)['input_ids']

    # 4. 拼接成完整的 input_ids
    input_ids = prompt_ids + response_ids
    # 注意：Qwen2 的 attention_mask 全是 1 即可，因为我们会在 loss function 中通过-100的label来忽略prompt部分
    attention_mask = [1] * len(input_ids)
    
    # 5. 构建 labels，提问部分用 -100 忽略
    labels = [-100] * len(prompt_ids) + response_ids
    
    # 6. 截断 (Truncation)
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
        
    # 7. 填充 (Padding)
    padding_length = MAX_LENGTH - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + [tokenizer.pad_token_id] * padding_length
        attention_mask = attention_mask + [0] * padding_length
        labels = labels + [-100] * padding_length
        
    return input_ids, attention_mask, labels

formatted_dataset = alpaca_data.map(
    operations=[process_func],
    input_columns=['instruction', 'input', 'output'], 
    output_columns=["input_ids", "attention_mask", "labels"],
    num_parallel_workers=16
)
print(formatted_dataset.get_col_names())

for input_ids, attention_mask, labels in formatted_dataset.create_tuple_iterator():
    print("input_ids:\n",input_ids)
    print("attention_mask:\n",attention_mask)
    print("labels:\n",labels)
    print("decoded input id:", tokenizer.decode(input_ids))
    break


['attention_mask', 'labels', 'input_ids']
input_ids:
 [1 1 1 ... 0 0 0]
attention_mask:
 [-100 -100 -100 ... -100 -100 -100]
labels:
 [151644   8948    198 ... 151643 151643 151643]
decoded input id: """""""""""""""""""""""""""""!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

## 模型微调
LoRA（Low-Rank Adaptation） 是一种高效微调大模型的技术，核心思想是冻结原始模型参数，通过向特定层注入低秩矩阵来实现参数更新，从而节省计算资源和内存。

我们需要在这里先定义 LoRA 的配置，并根据配置为模型添加 LoRA 部分

In [36]:
from mindnlp.peft import LoraConfig, TaskType, get_peft_model

# 配置LoRA
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    # 指定需要加入LoRA模块的网络层
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 设置训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1  # Dropout 比例
)
print("LoraConfig:\n",config)
# 根据上述的lora配置，为模型添加lora部分
model = get_peft_model(model, config)

LoraConfig:
 LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, base_model_name_or_path=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'o_proj', 'v_proj', 'down_proj', 'up_proj', 'gate_proj', 'k_proj', 'q_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)


接下来需要定义一个训练的超参数（可以理解为是训练本身的一种配置）

In [37]:
from mindnlp.engine import TrainingArguments, Trainer

# 定义训练超参数
args = TrainingArguments(
    # 输出保存路径
    output_dir="./output/qwen2.5-7B",
    # 训练批大小
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    # 每100个step打印一次日志
    logging_steps=100,
    # # 训练epoch数
    num_train_epochs=3600,
    # 培训使用：设置最大训练步数，训练到3600步时会自动停止
    max_steps=3600,

    # 模型权重保存长，1000个step保存一次模型权重
    save_steps=500,
    # 学习率
    learning_rate=1e-4
)

In [38]:
from mindnlp.engine.callbacks import TrainerCallback, TrainerState, TrainerControl
import os
PREFIX_CHECKPOINT_DIR = "checkpoint"
SAFE_WEIGHTS_NAME = "safe_model_qat.bin"
class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(
            args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}"
        )       

        # 保存adapter权重
        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path, safe_serialization=True)

        # remove base model safetensors to free more space
        base_model_path = os.path.join(checkpoint_folder, SAFE_WEIGHTS_NAME)
        os.remove(base_model_path) if os.path.exists(base_model_path) else None

        return control


In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=formatted_dataset,
    callbacks=[SavePeftModelCallback],
)
trainer.train()




  0%|          | 0/3600 [00:00<?, ?it/s]


  0%|          | 1/3600 [00:07<7:17:06,  7.29s/it]


  0%|          | 2/3600 [00:14<7:10:32,  7.18s/it]


  0%|          | 3/3600 [00:21<7:05:30,  7.10s/it]


  0%|          | 4/3600 [00:28<7:03:59,  7.07s/it]


  0%|          | 5/3600 [00:35<7:02:55,  7.06s/it]


  0%|          | 6/3600 [00:42<7:01:21,  7.03s/it]


  0%|          | 7/3600 [00:49<7:00:25,  7.02s/it]


  0%|          | 8/3600 [00:56<7:00:04,  7.02s/it]


  0%|          | 9/3600 [01:03<6:59:22,  7.01s/it]


  0%|          | 10/3600 [01:10<6:58:20,  6.99s/it]

等待一段时间，大语言模型就可以完成微调，这个时候就可以实现对话的能力。